In [7]:
!pip install -q snowflake-connector-python

In [8]:
import snowflake.connector
import pandas as pd
import requests, json
from requests.adapters import HTTPAdapter, Retry

In [9]:
connection_id = "33b779c7-54f4-43cf-b881-1744ea1b48a6"
storage_account = "onelakedb2"
catalog = 'ONELAKE'
DB = 'ADLS'
lakehouse_id = notebookutils.lakehouse.get("snowdata").id
workspace_id = notebookutils.runtime.context['currentWorkspaceId']

In [10]:
import duckdb
# this is a hack, use proper key vault to store credential
secret_path  = 'abfss://e9e96ee0-a31e-4b5f-9cdc-553e61d380e8@msit-onelake.dfs.fabric.microsoft.com/abaa4b17-70dc-475b-8ecd-e99612b6e1d3/Files/store/variable.ini'
kv =duckdb.sql(f""" select trim(split_part("[myvars]",' : ', 1) ) as var, trim(split_part("[myvars]",' : ', 2)) as values from read_csv('{secret_path}') """)
kv_values=duckdb.sql( " from kv where var in ('user' ,'password','account_snwo') ").fetchall()
for key, value in kv_values:
        globals()[key] = value

In [11]:
conn = snowflake.connector.connect(
    user               =  user,
    password           =  password,
    account            = account_snwo, 
    insecure_mode      = True
)
snow = conn.cursor()
snow.execute(f""" use {catalog}.{DB} """)
df =snow.execute(f""" SELECT table_name  FROM INFORMATION_SCHEMA.TABLES WHERE table_schema = '{DB}' AND IS_ICEBERG = 'YES';""").fetch_pandas_all()
results = []
for tbl in df['TABLE_NAME']:
  tbl_info_df = snow.execute(f""" CALL SYSTEM$GET_ICEBERG_TABLE_INFORMATION ('{tbl}')""").fetch_pandas_all()
  tbl_info_df['name'] = tbl
  results.append(tbl_info_df)
results_df = pd.concat(results, ignore_index=True)

results_df['location'] = results_df['SYSTEM$GET_ICEBERG_TABLE_INFORMATION'].apply(
        lambda x: json.loads(x).get('metadataLocation', '') # Parse JSON and get metadataLocation
    ).apply(
        lambda s:
            s.split('/metadata/')[0]       # Get the part before /metadata/ (e.g., abfss://.../duid)
            .replace('abfss://', '')       # Remove 'abfss://'
            .split('.net/', 1)[-1] + '/'   # Split by '.net/' and take the second part to get 'data/respo/duid' then add back trailing slash
            if '/metadata/' in s and s else '' # Apply logic only if string is not empty and contains /metadata/
    )
results_df = results_df[['name', 'location']]
display(results_df)
conn.close()

In [12]:
def invoke_fabric_api_request(method, uri, payload = None):
    
    API_ENDPOINT = "api.fabric.microsoft.com/v1"

    headers = {
        "Authorization": "Bearer " + notebookutils.credentials.getToken("pbi"),
        "Content-Type": "application/json"
    }

    try:
        url = f"https://{API_ENDPOINT}/{uri}"
            
        session = requests.Session()
        retries = Retry(total=3, backoff_factor=5, status_forcelist=[502, 503, 504])
        adapter = HTTPAdapter(max_retries=3)
        session.mount('http://', adapter)
        session.mount('https://', adapter)

        response = session.request(method, url, headers=headers, json=payload, timeout=240)

        if response.text is not None:
            response_text_cleaned = response.text.replace('\\r\\n', '')

            response_details = {
                'status_code': response.status_code,
                'response': response_text_cleaned,
                'headers': dict(response.headers.items()) 
            }
            print(json.dumps(response_details, indent=2))

    except requests.RequestException as ex:
        print(ex)
method = "post"
shortcut_conflict_policy = "Abort"
uri = f"workspaces/{workspace_id}/items/{lakehouse_id}/shortcuts?shortcutConflictPolicy={shortcut_conflict_policy}"

for index, row in results_df.iterrows():
    payload = {
    "path": "Tables",
    "name": row['name'],
    "target": {
        "adlsGen2": {
        "location": f"https://{storage_account}.dfs.core.windows.net/",
        "subpath": row['location'],
        "connectionId": connection_id
        }
    }
    }

    invoke_fabric_api_request(method, uri, payload)

{
  "status_code": 409,
  "response": "{\"requestId\":\"fd6811de-fa9e-416d-8458-1ebe8e38a9b1\",\"errorCode\":\"EntityConflict\",\"moreDetails\":[{\"errorCode\":\"ShorcutsOperationNotAllowed\",\"message\":\"Shortcut operation set to abort and shortcut with same name already exists.\"}],\"message\":\"The request could not be completed due to a conflict with an existing resource\"}",
  "headers": {
    "Cache-Control": "no-store, must-revalidate, no-cache",
    "Pragma": "no-cache",
    "Transfer-Encoding": "chunked",
    "Content-Type": "application/json; charset=utf-8",
    "x-ms-public-api-error-code": "EntityConflict",
    "Strict-Transport-Security": "max-age=31536000; includeSubDomains",
    "X-Frame-Options": "deny",
    "X-Content-Type-Options": "nosniff",
    "RequestId": "fd6811de-fa9e-416d-8458-1ebe8e38a9b1",
    "Access-Control-Expose-Headers": "RequestId",
    "request-redirected": "true",
    "home-cluster-uri": "https://df-msit-scus-redirect.analysis.windows.net/",
    "Dat